[View in Colaboratory](https://colab.research.google.com/github/RayleighKim/M11_Intro_To_Unsuerpvised_Learning/blob/master/11_1_Intro_to_Unsupervised_Learning_I.ipynb)

# Intro to Unsupervised Learning

## Text Retrieval Problem

### Your name :     

### Data Link :
* [DBpedia](https://wiki.dbpedia.org/)
* [Dataset](https://media.githubusercontent.com/media/RayleighKim/Example_datasets/master/People_wiki.csv)

#### 실습목표 I<br>
1. TF-IDF를 이해한다.
2. Scikit-Learn을 이용하여 클러스터링을 할 줄 안다.

#### 실습목표 II<br>
1. 문제의 구조를 정교화 하기 위해 발전한 과정의 흐름을 안다.
---------------
Rayleigh Kim @ D:plus


In [1]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
'''
사이킷런의 라이브러리들을 미리 불러두는 것도 좋지만,
세부적인 메쏘드가 너무 많아서 필요한것만 불러오도록 할 것이다.
'''


'\n사이킷런의 라이브러리들을 미리 불러두는 것도 좋지만,\n세부적인 메쏘드가 너무 많아서 필요한것만 불러오도록 할 것이다.\n'

## Load and Prepare the data

In [0]:
data_path = 'https://media.githubusercontent.com/media/RayleighKim/Example_datasets/master/People_wiki.csv'
people = pd.read_csv(data_path)

In [3]:
len(people)

59071

In [4]:
people.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


우리는 URL은 안쓸꺼니까! 날려버리자.

왜인지 URI로 되어 있다.

In [5]:
'name' in list(people)

True

In [6]:
if 'URI' in list(people):
    people.drop(labels=['URI'], axis=1, inplace=True)
people.head()

,name,text
0,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,G-Enka,henry krvits born 30 december 1974 in tallinn ...


## Explore the dataset

엘튼 존! 조지 클루니!

In [7]:
elton = people.loc[people['name']=='Elton John']
elton

,name,text
19923,Elton John,sir elton hercules john cbe born reginald kenn...


In [8]:
list(elton['text'])

['sir elton hercules john cbe born reginald kenneth dwight 25 march 1947 is an english singer songwriter composer pianist record producer and occasional actor he has worked with lyricist bernie taupin as his songwriter partner since 1967 they have collaborated on more than 30 albums to datein his fivedecade career elton john has sold more than 300 million records making him one of the bestselling music artists in the world he has more than fifty top 40 hits including seven consecutive no 1 us albums 58 billboard top 40 singles 27 top 10 four no 2 and nine no 1 for 31 consecutive years 19702000 he had at least one song in the billboard hot 100 his single something about the way you look tonightcandle in the wind 1997 sold over 33 million copies worldwide and is the bestselling single of all time he has received six grammy awards five brit awards winning two awards for outstanding contribution to music and the first brits icon in 2013 for his lasting impact on british culture an academy 

In [9]:
george = people.loc[people['name'].str.contains('George')]
george['name']

213                        George Brent Mickum IV
586                          George Athan Billias
977                            George M. Galambos
1100                                  George Azar
1323                                George Quasha
1402                       George de la Pe%C3%B1a
1418                                George E. Fox
1524                                Willie George
1710                                George Athans
2486                             George Halvorson
2654                               George Gabriel
2657                              Billy Georgette
2770                  George McMahon (politician)
2969                              George Brancato
3883                               George Kurdahi
4904                      George Hart (physicist)
5180                           George P. Fletcher
5624                                  George Piro
5918                             George P. Taylor
5952                                  George Roll


In [10]:
clooney = george.loc[george['name'].str.contains('Clooney')]
clooney

,name,text
38514,George Clooney,george timothy clooney born may 6 1961 is an a...


In [11]:
list(clooney['text'])

['george timothy clooney born may 6 1961 is an american actor writer producer director and activist he has received three golden globe awards for his work as an actor and two academy awards one for acting and the other for producingclooney made his acting debut on television in 1978 and later gained wide recognition in his role as dr doug ross on the longrunning medical drama er from 1994 to 1999 for which he received two emmy award nominations while working on er he began attracting a variety of leading roles in films including the superhero film batman robin 1997 and the crime comedy out of sight 1998 in which he first worked with a director who would become a longtime collaborator steven soderbergh in 1999 clooney took the lead role in three kings a wellreceived war satire set during the gulf warin 2001 clooneys fame widened with the release of his biggest commercial success the heist comedy oceans eleven the first of the film trilogy a remake of the 1960 film with frank sinatra as 

## Bag of Words Simple ver

텍스트 문서들은 있다.<br>

텍스트들을 단어 등장여부를 이용한 binary vector로 만들자.

In [0]:
'''
Library Loading
'''
from collections import Counter, OrderedDict
import sklearn.feature_extraction.text as text

### 문장 하나를 사전으로 만들어 보자.

In [0]:
def Ordered_words(id = 0):
    sample_string = people['text'][id]
    string_splitted = sample_string.split()
    word_dict = dict(Counter(string_splitted))
    word_dict_list = [(k,v) for k, v in word_dict.items()]
    word_dict_list.sort(key = lambda x:x[1], reverse=True)
    print('name : ',people['name'][id])
    print(sample_string)
    print(word_dict_list)
    

In [14]:
'''
Ordered_words(n)
n 을 바꾸어 가며 확인해보자.
'''

Ordered_words(3)

name :  Franz Rottensteiner
franz rottensteiner born in waidmannsfeld lower austria austria on 18 january 1942 is an austrian publisher and critic in the fields of science fiction and the fantasticrottensteiner studied journalism english and history at the university of vienna receiving his doctorate in 1969 he served about fifteen years as librarian and editor at the sterreichisches institut fr bauforschung in vienna in addition he produced a number of translations into german of leading sf authors including herbert w franke stanislaw lem philip k dick kobo abe cordwainer smith brian w aldiss and the strugatski brothersin 1973 his new york anthology view from another shore of european science fiction introduced a number of continental authors to the englishreading public some of the authors in the work are stanislaw lem josef nesvadba gerard klein and jeanpierre andrevonthe year 1975 saw the start of his series die phantastischen romane for seven years it republished works of both les

### 모든 텍스트를 이용하여, 존재하는 단어들의 사전을 만들어 보자!

In [0]:
word_bin = text.CountVectorizer(binary = True)

In [0]:
X = word_bin.fit_transform(people['text'])

In [27]:
type(X)

scipy.sparse.csr.csr_matrix

In [0]:
print(len(word_bin.vocabulary_))
word_bin.vocabulary_

### 문장 하나를 벡터로 만들어 보자!

 모든 문장을 이용하여 단어 사전을 만들어 두었다!
 
 이제, 문장 하나를 단어 존재 여부를 이용하여 벡터로 만들 수 있다.

In [0]:
def simple_BOW_vector(id=0):
    sample_array = word_bin.transform([people['text'][id]]).toarray()[0]
    print('array 사이즈 : ', len(sample_array))
    print('등장한 단어의 개수 : ', sum(sample_array==1))
    print(sample_array)

In [30]:
simple_BOW_vector(4)

array 사이즈 :  548429
등장한 단어의 개수 :  113
[0 0 0 ... 0 0 0]


### 문장들을 벡터로 만들어서 데이터 프레임에 붙일 수 있을까?

정말 궁금한 사람들은 아래의 코드를 실행해보자.

메모리가 터지는 것을 관찰할 수 있다!

```
word_bin.transform(people['text'].toarray())
```

위에서 아래와 같은 코드를 사용했었다.

```
X = word_bin.fit_transform(people['text'])
```

X는 scipy.sparse.csr.csr_matrix 라고 위에서 확인했었는데,  사실 X가 우리가 원하는 그 메트릭스다!

다만, 사이즈가 54만이 넘어가는 벡터에서 고작 단어 몇백개의 위치만 1이 되는 array를 그대로 생성하는 것은 메모리가 괴로워하기 때문에, 우리는 잘 압축된 메트릭스를 대신하여 사용한다.

아래의 셀을 실행해보면 어떻게 된 것인지 감을 잡을 수 있다!

In [0]:
print(X[0,:])

In [41]:
people.loc[0:1]

,name,text
0,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
